Reference: https://github.com/entbappy/NLP-Projects-Notebooks/blob/master/Text-Summarization-Fine-tuning-Transformers-model.ipynb

# Fine Tune

In [ ]:
dfmix = df[['id','dialogue','summary']]

In [ ]:
dfmix

,id,dialogue,summary
0,0,Amanda: I baked cookies. Do you want some?\r\...,Amanda baked cookies and will bring Jerry some...
1,1,Olivia: Who are you voting for in this electio...,Olivia and Oliver both plan to vote for the Li...
2,2,"Tim: Hi, what's up?\r\nKim: Bad mood tbh, I wa...",Kim may try the pomodoro technique recommended...
3,3,"Edward: Rachel, I think I'm in ove with Bella....",Rachel abruptly interrupted Edward's confessio...
4,4,Sam: hey overheard rick say something\r\nSam:...,"Sam is confused, because he overheard Rick com..."
...,...,...,...
14727,14727,Romeo: You are on my ‘People you may know’ lis...,Romeo reached out to Greta as a potential frie...
14728,14728,Theresa: <file_photo>\r\nTheresa: <file_photo>...,Theresa is at work. She gets free food and fre...
14729,14729,John: Every day some bad news. Japan will hunt...,John expressed his outrage over Japan's decisi...
14730,14730,Jennifer: Dear Celia! How are you doing?\r\nJe...,Celia couldn't make it to the afternoon with t...


In [ ]:
dataset_samsum

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [ ]:
from datasets import Dataset
dataset = Dataset.from_pandas(dfmix)

In [ ]:
dataset

Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 14732
})

In [ ]:
val_dataset = dataset_samsum['validation']

In [ ]:
val_dataset

Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 818
})

In [ ]:
import datasets
dd = datasets.DatasetDict({"train":dataset,"validation":val_dataset})

In [ ]:
dd

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [ ]:
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer_large(example_batch['dialogue'] , max_length = 1024, truncation = True )
    
    with tokenizer_large.as_target_tokenizer():
        target_encodings = tokenizer_large(example_batch['summary'], max_length = 128, truncation = True )
        
    return {
        'input_ids' : input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }
    
dataset_samsum_pt = dd.map(convert_examples_to_features, batched = True)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

In [ ]:
#dataset_val[0]

In [ ]:
from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer_large, model=model_large)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Project/Fine_Tuning

/content/drive/MyDrive/Project/Fine_Tuning


In [ ]:
from transformers import TrainingArguments, Trainer

trainer_args = TrainingArguments(
    output_dir='bart-samsum-half', num_train_epochs=25, warmup_steps=500,
    per_device_train_batch_size=4, per_device_eval_batch_size=4,
    weight_decay=0.01, logging_steps=10,
    evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
    gradient_accumulation_steps=16
) 
     

In [ ]:
trainer = Trainer(model=model_large, args=trainer_args,
                  tokenizer=tokenizer_large, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt['train'], 
                  eval_dataset=dataset_samsum_pt['validation'])

In [ ]:
import gc

gc.collect()
torch.cuda.empty_cache()

In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
500,1.087400,1.478014
1000,0.684100,1.658565
1500,0.413300,1.965265
2000,0.251600,2.348899
2500,0.159100,2.617553
3000,0.089200,2.997341
3500,0.066100,3.016472
4000,0.047300,3.298780
4500,0.032900,3.306062


In [ ]:
model_large.save_pretrained("/content/drive/MyDrive/dl-gp/bart-half-3")
tokenizer_large.save_pretrained("/content/drive/MyDrive/dl-gp/bart-half-3")

NameError: ignored

In [ ]:
tokenizer_large = BartTokenizer.from_pretrained("/content/drive/MyDrive/dl-gp/bart-half-3")
model_large = BartForConditionalGeneration.from_pretrained("/content/drive/MyDrive/dl-gp/bart-half-3").to(device)

In [ ]:
trainer_args = TrainingArguments(
    output_dir='bart-samsum-half', num_train_epochs=1, warmup_steps=500,
    per_device_train_batch_size=8, per_device_eval_batch_size=4,
    weight_decay=0.01, logging_steps=10,
    evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
    gradient_accumulation_steps=16
) 
     

In [ ]:
trainer = Trainer(model=model_large, args=trainer_args,
                  tokenizer=tokenizer_large, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt['train'], 
                  eval_dataset=dataset_samsum_pt['validation'])

trainer.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=115, training_loss=1.0131884035856829, metrics={'train_runtime': 541.0638, 'train_samples_per_second': 27.228, 'train_steps_per_second': 0.213, 'total_flos': 1.0860563227017216e+16, 'train_loss': 1.0131884035856829, 'epoch': 1.0})